In [22]:
import os
from langchain.llms import GooglePalm
from langchain_experimental.sql import SQLDatabaseChain
from dotenv import load_dotenv

In [18]:
load_dotenv('.env')

google_api_key=os.getenv('GOOGLE_API_KEY')

In [19]:
palmLlm = GooglePalm(
    google_api_key=key,
    temperature=0.2
)

In [20]:
from langchain.utilities import SQLDatabase

db_user='root'
db_password=''
db_host='localhost'
db_name='atliq_tshirts'

sqlDb=SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(sqlDb.table_info)


CREATE TABLE discounts (
	discount_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER(11) NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id)
)COLLATE latin1_swedish_ci DEFAULT CHARSET=latin1 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount

*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER(11), 
	stock_quantity INTEGER(11) NOT NULL, 
	PRIMARY KEY (t_shirt_id)
)COLLATE latin1_swedish_ci DEFAULT CHARSET=latin1 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	XS	11	72
3	Adidas	White	XL	31	23
4	Van Huesen	Blue	M	12	11
*/


In [23]:
sqlChain=SQLDatabaseChain.from_llm(llm=palmLlm, db=sqlDb, verbose=True)

In [30]:
qns1 = sqlChain("How many t-shirts do we have left for nike in extra small size and white color?")

C:\Users\Pinil\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  def emit_warning() -> None:




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(40,)]
Answer:40
> Finished chain.


In [31]:
qns2 = sqlChain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('404'),)]
Answer:404
> Finished chain.


In [32]:
qns2 = sqlChain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('19027'),)]
Answer:19027
> Finished chain.


In [35]:
qns3 = sqlChain.run("If we have to sell all the Levi’s T-shirts today . How much revenue our store will generate ?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today . How much revenue our store will generate ?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE brand = 'Levi' AND stock_quantity > 0
SQLResult: [(Decimal('565'),)]
Answer:565
> Finished chain.


In [36]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = sqlChain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26871.000000'),)]
Answer:26871.0
> Finished chain.


In [37]:
qns4 = sqlChain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('26871'),)]
Answer:26871
> Finished chain.


In [38]:
qns5 = sqlChain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('200'),)]
Answer:200
> Finished chain.


# Few short learning

In [39]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [47]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

ImportError: cannot import name 'InMemoryByteStore' from 'langchain_core.stores' (C:\Users\Pinil\anaconda3\Lib\site-packages\langchain_core\stores.py)

In [48]:
from langchain.storage import InMemoryByteStore

ImportError: cannot import name 'InMemoryByteStore' from 'langchain_core.stores' (C:\Users\Pinil\anaconda3\Lib\site-packages\langchain_core\stores.py)

In [ ]:
%pip install --upgrade --quiet  langchain-openai faiss-cpu